In [ ]:
pip install plotly==5.6.0 pandas matplotlib squarify

In [ ]:
import requests
import json
import time
import numpy as np
import os

loc_percentages = []
loc = []
loc_committers = []
freq_committers = []
freq_committer_scores = []

def analyze_repo(repo_url):
    global loc_percentages
    global loc
    global loc_committers
    global freq_committers
    global freq_committer_scores
    api_url = os.getenv('REST_API_URL')  # "http://rest-api/repo-analysis/api/analysis"
    # repo_url = "https://github.com/apache/kafka"
    query_string = {'repo_url': repo_url}
    headers = {'Accept': 'application/json'}

    done = False
    while not done:
        response = requests.request("GET", api_url, headers=headers, params=query_string)
        res_obj = json.loads(response.text)
        if res_obj['status'] != 'done': 
            print('.', end='')
            time.sleep(5)
            continue
        else:
            print('')
            loc_percentages = [detail['loc_percentage'] for detail in res_obj['loc_analysis'].values()]
            loc = [detail['loc'] for detail in res_obj['loc_analysis'].values()]
            print(f"Top five loc_percentages={loc_percentages[:5]}")
            print(f"Top five loc={loc[:5]}")

            loc_committers = list(res_obj['loc_analysis'].keys())
            print(f"Top five committers={loc_committers[:5]}")
            
            freq_committers = list(res_obj['commit_freq_analysis'].keys())
            print(f"Top five frequent committers={freq_committers[:5]}")
            freq_committer_scores = list(res_obj['commit_freq_analysis'].values())
            print(f"Top five frequent committer scores={freq_committer_scores[:5]}")
            done = True

In [ ]:
import matplotlib.pyplot as plt

def plot_analysis(committers, committers_stats, repo_url, title):

    fig, ax = plt.subplots(figsize=(10, 10), subplot_kw=dict(aspect="equal"))

    wedges, texts = ax.pie(committers_stats, wedgeprops=dict(width=0.5), startangle=-40)

    bbox_props = dict(boxstyle="square,pad=0.3", fc="w", ec="k", lw=0.72)
    kw = dict(arrowprops=dict(arrowstyle="-"),
              bbox=bbox_props, zorder=0, va="center")

    for i, p in enumerate(wedges):
        ang = (p.theta2 - p.theta1)/2. + p.theta1
        y = np.sin(np.deg2rad(ang))
        x = np.cos(np.deg2rad(ang))
        horizontalalignment = {-1: "right", 1: "left"}[int(np.sign(x))]
        connectionstyle = "angle,angleA=0,angleB={}".format(ang)
        kw["arrowprops"].update({"connectionstyle": connectionstyle})
        ax.annotate(committers[i], xy=(x, y), xytext=(1.35*np.sign(x), 1.4*y),
                    horizontalalignment=horizontalalignment, **kw)

    ax.set_title(f"{title}\nRepo={repo_url}")

    plt.show()

In [ ]:
repo_url = 'https://github.com/axios/axios'

In [ ]:
analyze_repo(repo_url)

# Fetching data. This may take several minutes (10+) for big/popular repos (eg. Apache Kafka, Apache Druid).
Some suggestions are: https://github.com/search?q=stars%3A%22%3E+100%22+size%3A%3C5000&type=Repositories&ref=advsearch&l=&l=

# Who has committed most Lines of Code (loc) to master branch head ?

In [ ]:
plot_analysis(loc_committers, loc_percentages, repo_url, 'Distribution by loc% per committers for the master branch head')

# In the last one year who committed most frequently to the master branch ?

In [ ]:
plot_analysis(freq_committers, freq_committer_scores, repo_url, 'Distribution by consistency-score per committer for the master branch head')